In [16]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense




In [17]:
ds=pd.read_csv("credit_score_dataset_updated.csv")
df=pd.DataFrame(ds)
df.head()

,credit_score,years_with_bank,deposit_amount,repayment_history,market_trend,credit_score_adjustment,years_with_bank_adjustment,deposit_amount_adjustment,repayment_history_adjustment,market_trend_adjustment,final_interest_rate_adjustment_bps,adjusted_interest_rate,product_type
0,452,8,36221300,good,falling,-84,43,93,-19,-29,4,7.54,Home Loans
1,785,6,69293176,excellent,falling,91,33,93,62,-42,237,9.87,Fixed Deposits
2,620,0,29728001,excellent,stable,2,2,77,89,18,188,9.38,Fixed Deposits
3,456,6,13024131,excellent,falling,-63,48,90,85,-42,118,8.68,Home Loans
4,421,6,72719057,excellent,rising,-65,28,67,57,33,120,8.70,Fixed Deposits


In [18]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df['repayment_history']=le.fit_transform(df['repayment_history'])
df['market_trend']=le.fit_transform(df['market_trend'])
df['product_type']=le.fit_transform(df['product_type'])
df.head()

,credit_score,years_with_bank,deposit_amount,repayment_history,market_trend,credit_score_adjustment,years_with_bank_adjustment,deposit_amount_adjustment,repayment_history_adjustment,market_trend_adjustment,final_interest_rate_adjustment_bps,adjusted_interest_rate,product_type
0,452,8,36221300,1,0,-84,43,93,-19,-29,4,7.54,1
1,785,6,69293176,0,0,91,33,93,62,-42,237,9.87,0
2,620,0,29728001,0,2,2,2,77,89,18,188,9.38,0
3,456,6,13024131,0,0,-63,48,90,85,-42,118,8.68,1
4,421,6,72719057,0,1,-65,28,67,57,33,120,8.70,0


In [20]:
X = df.drop(columns=['adjusted_interest_rate'])  # Features
y = df['adjusted_interest_rate']  # Target variable
print(X.columns.tolist())
print(y.name)

['credit_score', 'years_with_bank', 'deposit_amount', 'repayment_history', 'market_trend', 'credit_score_adjustment', 'years_with_bank_adjustment', 'deposit_amount_adjustment', 'repayment_history_adjustment', 'market_trend_adjustment', 'final_interest_rate_adjustment_bps', 'product_type']
adjusted_interest_rate


In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardizing the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [22]:
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

In [24]:
model = Sequential([
    LSTM(50, activation='relu', return_sequences=True, input_shape=(1, X_train.shape[2])),
    LSTM(25, activation='relu'),
    Dense(10, activation='relu'),
    Dense(1)  # Output layer for regression
])

In [25]:
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [26]:
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/50
250/250 [==============================] - 13s 17ms/step - loss: 28.1646 - mae: 4.1618 - val_loss: 2.7521 - val_mae: 1.3355
Epoch 2/50
250/250 [==============================] - 3s 11ms/step - loss: 1.4518 - mae: 0.9479 - val_loss: 0.6815 - val_mae: 0.6654
Epoch 3/50
250/250 [==============================] - 3s 12ms/step - loss: 0.3832 - mae: 0.4856 - val_loss: 0.2152 - val_mae: 0.3697
Epoch 4/50
250/250 [==============================] - 3s 10ms/step - loss: 0.1530 - mae: 0.3079 - val_loss: 0.1104 - val_mae: 0.2632
Epoch 5/50
250/250 [==============================] - 5s 20ms/step - loss: 0.0846 - mae: 0.2276 - val_loss: 0.0700 - val_mae: 0.2077
Epoch 6/50
250/250 [==============================] - 8s 30ms/step - loss: 0.0545 - mae: 0.1792 - val_loss: 0.0467 - val_mae: 0.1656
Epoch 7/50
250/250 [==============================] - 3s 11ms/step - loss: 0.0412 - mae: 0.1524 - val_loss: 0.0379 - val_mae: 0.1457
Epoch 8/50
250/250 [==============================] - 3s 12ms/step 

In [27]:
from sklearn.metrics import r2_score
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Calculate R² scores
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

print(f"R² Score (Training): {r2_train}")
print(f"R² Score (Testing): {r2_test}")

63/63 [==============================] - 2s 7ms/step
R² Score (Training): 0.9845777451544806
R² Score (Testing): 0.9795825690815024
